Deep Learning
=============

Assignment 3
------------

Previously in `2_fullyconnected.ipynb`, you trained a logistic regression and a neural network model.

The goal of this assignment is to explore regularization techniques.

In [1]:
# These are all the modules we'll be using later. Make sure you can import them
# before proceeding further.
from __future__ import print_function
import numpy as np
import tensorflow as tf
from six.moves import cPickle as pickle

First reload the data we generated in `1_notmnist.ipynb`.

In [2]:
pickle_file = 'notMNIST.pickle'

with open(pickle_file, 'rb') as f:
  save = pickle.load(f)
  train_dataset = save['train_dataset']
  train_labels = save['train_labels']
  valid_dataset = save['valid_dataset']
  valid_labels = save['valid_labels']
  test_dataset = save['test_dataset']
  test_labels = save['test_labels']
  del save  # hint to help gc free up memory
  print('Training set', train_dataset.shape, train_labels.shape)
  print('Validation set', valid_dataset.shape, valid_labels.shape)
  print('Test set', test_dataset.shape, test_labels.shape)

Training set (200000, 28, 28) (200000,)
Validation set (10000, 28, 28) (10000,)
Test set (10000, 28, 28) (10000,)


Reformat into a shape that's more adapted to the models we're going to train:
- data as a flat matrix,
- labels as float 1-hot encodings.

In [3]:
image_size = 28
num_labels = 10

def reformat(dataset, labels):
  dataset = dataset.reshape((-1, image_size * image_size)).astype(np.float32)
  # Map 1 to [0.0, 1.0, 0.0 ...], 2 to [0.0, 0.0, 1.0 ...]
  labels = (np.arange(num_labels) == labels[:,None]).astype(np.float32)
  return dataset, labels
train_dataset, train_labels = reformat(train_dataset, train_labels)
valid_dataset, valid_labels = reformat(valid_dataset, valid_labels)
test_dataset, test_labels = reformat(test_dataset, test_labels)
print('Training set', train_dataset.shape, train_labels.shape)
print('Validation set', valid_dataset.shape, valid_labels.shape)
print('Test set', test_dataset.shape, test_labels.shape)

Training set (200000, 784) (200000, 10)
Validation set (10000, 784) (10000, 10)
Test set (10000, 784) (10000, 10)


In [4]:
def accuracy(predictions, labels):
  return (100.0 * np.sum(np.argmax(predictions, 1) == np.argmax(labels, 1))
          / predictions.shape[0])

---
Problem 1
---------

Introduce and tune L2 regularization for both logistic and neural network models. Remember that L2 amounts to adding a penalty on the norm of the weights to the loss. In TensorFlow, you can compute the L2 loss for a tensor `t` using `nn.l2_loss(t)`. The right amount of regularization should improve your validation / test accuracy.

---

In [10]:
# With gradient descent training, even this much data is prohibitive.
# Subset the training data for faster turnaround.
train_subset = 10000

graph = tf.Graph()
with graph.as_default():

  # Input data.
  # Load the training, validation and test data into constants that are
  # attached to the graph.
  tf_train_dataset = tf.constant(train_dataset[:train_subset, :])
  tf_train_labels = tf.constant(train_labels[:train_subset])
  tf_valid_dataset = tf.constant(valid_dataset)
  tf_test_dataset = tf.constant(test_dataset)
  
  # Variables.
  # These are the parameters that we are going to be training. The weight
  # matrix will be initialized using random values following a (truncated)
  # normal distribution. The biases get initialized to zero.
  weights = tf.Variable(
    tf.truncated_normal([image_size * image_size, num_labels]))
  biases = tf.Variable(tf.zeros([num_labels]))
  
  # Training computation.
  # We multiply the inputs with the weight matrix, and add biases. We compute
  # the softmax and cross-entropy (it's one operation in TensorFlow, because
  # it's very common, and it can be optimized). We take the average of this
  # cross-entropy across all training examples: that's our loss.
  logits = tf.matmul(tf_train_dataset, weights) + biases
  
  loss = tf.reduce_mean(
    tf.nn.softmax_cross_entropy_with_logits(labels=tf_train_labels, logits=logits) + 0.01*tf.nn.l2_loss(weights))
  
  # Optimizer.
  # We are going to find the minimum of this loss using gradient descent.
  optimizer = tf.train.GradientDescentOptimizer(0.5).minimize(loss)
  
  # Predictions for the training, validation, and test data.
  # These are not part of training, but merely here so that we can report
  # accuracy figures as we train.
  train_prediction = tf.nn.softmax(logits)
  valid_prediction = tf.nn.softmax(
    tf.matmul(tf_valid_dataset, weights) + biases)
  test_prediction = tf.nn.softmax(tf.matmul(tf_test_dataset, weights) + biases)

In [11]:
num_steps = 801

with tf.Session(graph=graph) as session:
  # This is a one-time operation which ensures the parameters get initialized as
  # we described in the graph: random weights for the matrix, zeros for the
  # biases. 
  tf.global_variables_initializer().run()
  print('Initialized')
  for step in range(num_steps):
    # Run the computations. We tell .run() that we want to run the optimizer,
    # and get the loss value and the training predictions returned as numpy
    # arrays.
    _, l, predictions = session.run([optimizer, loss, train_prediction])
    if (step % 100 == 0):
      print('Loss at step %d: %f' % (step, l))
      print('Training accuracy: %.1f%%' % accuracy(
        predictions, train_labels[:train_subset, :]))
      # Calling .eval() on valid_prediction is basically like calling run(), but
      # just to get that one numpy array. Note that it recomputes all its graph
      # dependencies.
      print('Validation accuracy: %.1f%%' % accuracy(
        valid_prediction.eval(), valid_labels))
  print('Test accuracy: %.1f%%' % accuracy(test_prediction.eval(), test_labels))

Initialized
Loss at step 0: 49.397606
Training accuracy: 7.0%
Validation accuracy: 8.6%
Loss at step 100: 11.828752
Training accuracy: 74.0%
Validation accuracy: 72.1%
Loss at step 200: 4.482669
Training accuracy: 79.1%
Validation accuracy: 76.8%
Loss at step 300: 1.975801
Training accuracy: 82.3%
Validation accuracy: 79.7%
Loss at step 400: 1.123369
Training accuracy: 83.8%
Validation accuracy: 81.3%
Loss at step 500: 0.830524
Training accuracy: 84.3%
Validation accuracy: 81.7%
Loss at step 600: 0.728721
Training accuracy: 84.5%
Validation accuracy: 82.0%
Loss at step 700: 0.692995
Training accuracy: 84.5%
Validation accuracy: 82.0%
Loss at step 800: 0.680362
Training accuracy: 84.5%
Validation accuracy: 82.0%
Test accuracy: 88.6%


In [7]:
n_hidden = 1024
n_input = 784 # MNIST data input (img shape: 28*28)
learning_rate = 0.001
training_epochs = 10
display_step = 2
batch_size = 128

def multilayer_perceptron(x, weights, biases):  
    layer_1 = tf.add(tf.matmul(x, weights['h1']), biases['b1'])
    layer_1 = tf.nn.relu(layer_1)
    out_layer = tf.matmul(layer_1, weights['out']) + biases['out']
    return out_layer

graph = tf.Graph()
with graph.as_default():


  # Input data. For the training data, we use a placeholder that will be fed
  # at run time with a training minibatch.
  tf_train_dataset = tf.placeholder(tf.float32,
                                    shape=(batch_size, image_size * image_size))
  tf_train_labels = tf.placeholder(tf.float32, shape=(batch_size, num_labels))
  tf_valid_dataset = tf.constant(valid_dataset)
  tf_test_dataset = tf.constant(test_dataset)
    

  weights = {
      'h1': tf.Variable(tf.random_normal([n_input, n_hidden])),
      'out': tf.Variable(tf.random_normal([n_hidden, num_labels]))
  }
  biases = {
      'b1': tf.Variable(tf.random_normal([n_hidden])),
      'out': tf.Variable(tf.random_normal([num_labels]))
  }
  pred = multilayer_perceptron(tf_train_dataset, weights, biases)
  # Define loss and optimizer
  cost = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(logits=pred, labels=tf_train_labels)+ 0.001*tf.nn.l2_loss(weights['out'])+0.001*tf.nn.l2_loss(weights['h1']))
  optimizer = tf.train.AdamOptimizer(learning_rate=learning_rate).minimize(cost)
    
  train_prediction = tf.nn.softmax(pred)
  valid_prediction = tf.nn.softmax(multilayer_perceptron(tf_valid_dataset, weights, biases))
  test_prediction = tf.nn.softmax(multilayer_perceptron(tf_test_dataset, weights, biases))


In [8]:
with tf.Session(graph=graph) as sess:
    tf.global_variables_initializer().run()
    # Training cycle
    for epoch in range(training_epochs):
        avg_cost = 0.
        num_steps = train_labels.shape[0]//batch_size
        # Loop over all batches
        for step in range(num_steps):
            offset = (step * batch_size) % (train_labels.shape[0] - batch_size)
            # Generate a minibatch.
            batch_data = train_dataset[offset:(offset + batch_size), :]
            batch_labels = train_labels[offset:(offset + batch_size), :]
    
            # Run optimization op (backprop) and cost op (to get loss value)
            _, c, prediction = sess.run([optimizer, cost, train_prediction], feed_dict={tf_train_dataset: batch_data,
                                                          tf_train_labels: batch_labels})
            # Compute average loss
            avg_cost += c / num_steps
        # Display logs per epoch step
        if epoch % display_step == 0:
            print("Epoch:", '%04d' % (epoch+1), "cost=", \
                "{:.9f}".format(avg_cost))
            print("Minibatch accuracy: %.1f%%" % accuracy(prediction, batch_labels))
            print("Validation accuracy: %.1f%%" % accuracy(valid_prediction.eval(), valid_labels))
            print("Test accuracy: %.1f%%" % accuracy(test_prediction.eval(), test_labels))
    print("Optimization Finished!")


Epoch: 0001 cost= 413.918739544
Minibatch accuracy: 85.9%
Validation accuracy: 81.0%
Test accuracy: 88.4%
Epoch: 0003 cost= 256.803128661
Minibatch accuracy: 85.2%
Validation accuracy: 83.2%
Test accuracy: 90.0%
Epoch: 0005 cost= 151.564738822
Minibatch accuracy: 85.9%
Validation accuracy: 84.3%
Test accuracy: 90.9%
Epoch: 0007 cost= 84.064514004
Minibatch accuracy: 89.8%
Validation accuracy: 83.9%
Test accuracy: 90.4%
Epoch: 0009 cost= 43.978675693
Minibatch accuracy: 93.0%
Validation accuracy: 84.9%
Test accuracy: 90.9%
Optimization Finished!


---
Problem 2
---------
Let's demonstrate an extreme case of overfitting. Restrict your training data to just a few batches. What happens?

---

In [40]:
batch_size = 128
with tf.Session(graph=graph) as sess:
    tf.global_variables_initializer().run()
    # Training cycle
    for epoch in range(training_epochs):
        avg_cost = 0.
        num_steps = 10
        # Loop over all batches
        for step in range(num_steps):
            offset = (step * batch_size) % (256 - batch_size)
            # Generate a minibatch.
            batch_data = train_dataset[offset:(offset + batch_size), :]
            batch_labels = train_labels[offset:(offset + batch_size), :]
    
            # Run optimization op (backprop) and cost op (to get loss value)
            _, c, prediction = sess.run([optimizer, cost, train_prediction], feed_dict={tf_train_dataset: batch_data,
                                                          tf_train_labels: batch_labels})
            # Compute average loss
            avg_cost += c / num_steps
        # Display logs per epoch step
        if epoch % display_step == 0:
            print("Epoch:", '%04d' % (epoch+1), "cost=", \
                "{:.9f}".format(avg_cost))
            print("Minibatch accuracy: %.1f%%" % accuracy(prediction, batch_labels))
            print("Validation accuracy: %.1f%%" % accuracy(valid_prediction.eval(), valid_labels))
            print("Test accuracy: %.1f%%" % accuracy(test_prediction.eval(), test_labels))
    print("Optimization Finished!")

Epoch: 0001 cost= 854.439764404
Minibatch accuracy: 21.1%
Validation accuracy: 21.0%
Test accuracy: 22.6%
Epoch: 0003 cost= 532.109420776
Minibatch accuracy: 56.2%
Validation accuracy: 47.2%
Test accuracy: 52.0%
Epoch: 0005 cost= 449.733944702
Minibatch accuracy: 71.1%
Validation accuracy: 56.1%
Test accuracy: 62.1%
Epoch: 0007 cost= 421.723587036
Minibatch accuracy: 85.2%
Validation accuracy: 58.6%
Test accuracy: 64.5%
Epoch: 0009 cost= 414.386346436
Minibatch accuracy: 91.4%
Validation accuracy: 60.2%
Test accuracy: 65.8%
Optimization Finished!


**Answer: ** I only train the first 128 data from the training set. The training accuracy is 91% and test accuracy is only 66%, clearly an overfitting.

---
Problem 3
---------
Introduce Dropout on the hidden layer of the neural network. Remember: Dropout should only be introduced during training, not evaluation, otherwise your evaluation results would be stochastic as well. TensorFlow provides `nn.dropout()` for that, but you have to make sure it's only inserted during training.

What happens to our extreme overfitting case?

---

In [25]:
dropout = 0.75

def multilayer_perceptron_dropout(x, weights, biases):  
    layer_1 = tf.add(tf.matmul(x, weights['h1']), biases['b1'])
    layer_1 = tf.nn.relu(layer_1)
    layer_1 = tf.nn.dropout(layer_1, dropout)
    out_layer = tf.matmul(layer_1, weights['out']) + biases['out']
    return out_layer

graph = tf.Graph()
with graph.as_default():


  # Input data. For the training data, we use a placeholder that will be fed
  # at run time with a training minibatch.
  tf_train_dataset = tf.placeholder(tf.float32,
                                    shape=(batch_size, image_size * image_size))
  tf_train_labels = tf.placeholder(tf.float32, shape=(batch_size, num_labels))
  tf_valid_dataset = tf.constant(valid_dataset)
  tf_test_dataset = tf.constant(test_dataset)
    

  weights = {
      'h1': tf.Variable(tf.random_normal([n_input, n_hidden])),
      'out': tf.Variable(tf.random_normal([n_hidden, num_labels]))
  }
  biases = {
      'b1': tf.Variable(tf.random_normal([n_hidden])),
      'out': tf.Variable(tf.random_normal([num_labels]))
  }
  pred = multilayer_perceptron_dropout(tf_train_dataset, weights, biases)
  # Define loss and optimizer
  cost = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(logits=pred, labels=tf_train_labels)+ 0.001*tf.nn.l2_loss(weights['out'])+0.001*tf.nn.l2_loss(weights['h1']))
  optimizer = tf.train.AdamOptimizer(learning_rate=learning_rate).minimize(cost)
    
  train_prediction = tf.nn.softmax(pred)
  valid_prediction = tf.nn.softmax(multilayer_perceptron(tf_valid_dataset, weights, biases))
  test_prediction = tf.nn.softmax(multilayer_perceptron(tf_test_dataset, weights, biases))




In [26]:
with tf.Session(graph=graph) as sess:
    tf.global_variables_initializer().run()
    # Training cycle
    for epoch in range(training_epochs):
        avg_cost = 0.
        num_steps = train_labels.shape[0]//batch_size
        # Loop over all batches
        for step in range(num_steps):
            offset = (step * batch_size) % (train_labels.shape[0] - batch_size)
            # Generate a minibatch.
            batch_data = train_dataset[offset:(offset + batch_size), :]
            batch_labels = train_labels[offset:(offset + batch_size), :]
    
            # Run optimization op (backprop) and cost op (to get loss value)
            _, c, prediction = sess.run([optimizer, cost, train_prediction], feed_dict={tf_train_dataset: batch_data,
                                                          tf_train_labels: batch_labels})
            # Compute average loss
            avg_cost += c / num_steps
        # Display logs per epoch step
        if epoch % display_step == 0:
            print("Epoch:", '%04d' % (epoch+1), "cost=", \
                "{:.9f}".format(avg_cost))
            print("Minibatch accuracy: %.1f%%" % accuracy(prediction, batch_labels))
            print("Validation accuracy: %.1f%%" % accuracy(valid_prediction.eval(), valid_labels))
            print("Test accuracy: %.1f%%" % accuracy(test_prediction.eval(), test_labels))
    print("Optimization Finished!")



Epoch: 0001 cost= 445.275814862
Minibatch accuracy: 81.2%
Validation accuracy: 83.2%
Test accuracy: 89.9%
Epoch: 0003 cost= 276.768726246
Minibatch accuracy: 89.1%
Validation accuracy: 84.8%
Test accuracy: 91.4%
Epoch: 0005 cost= 158.581448104
Minibatch accuracy: 85.2%
Validation accuracy: 85.3%
Test accuracy: 91.2%
Epoch: 0007 cost= 77.958313916
Minibatch accuracy: 89.1%
Validation accuracy: 85.3%
Test accuracy: 91.5%
Epoch: 0009 cost= 31.628549851
Minibatch accuracy: 85.9%
Validation accuracy: 85.8%
Test accuracy: 91.9%
Optimization Finished!


** Answer :** Test accuracy increased by 0.1% from dropout.

In [41]:
with tf.Session(graph=graph) as sess:
    tf.global_variables_initializer().run()
    # Training cycle
    for epoch in range(training_epochs):
        avg_cost = 0.
        num_steps = 10
        # Loop over all batches
        for step in range(num_steps):
            offset = (step * batch_size) % (256 - batch_size)
            # Generate a minibatch.
            batch_data = train_dataset[offset:(offset + batch_size), :]
            batch_labels = train_labels[offset:(offset + batch_size), :]
    
            # Run optimization op (backprop) and cost op (to get loss value)
            _, c, prediction = sess.run([optimizer, cost, train_prediction], feed_dict={tf_train_dataset: batch_data,
                                                          tf_train_labels: batch_labels})
            # Compute average loss
            avg_cost += c / num_steps
        # Display logs per epoch step
        if epoch % display_step == 0:
            print("Epoch:", '%04d' % (epoch+1), "cost=", \
                "{:.9f}".format(avg_cost))
            print("Minibatch accuracy: %.1f%%" % accuracy(prediction, batch_labels))
            print("Validation accuracy: %.1f%%" % accuracy(valid_prediction.eval(), valid_labels))
            print("Test accuracy: %.1f%%" % accuracy(test_prediction.eval(), test_labels))
    print("Optimization Finished!")

Epoch: 0001 cost= 733.338946533
Minibatch accuracy: 22.7%
Validation accuracy: 24.8%
Test accuracy: 26.6%
Epoch: 0003 cost= 495.586004639
Minibatch accuracy: 67.2%
Validation accuracy: 48.8%
Test accuracy: 53.1%
Epoch: 0005 cost= 439.221804810
Minibatch accuracy: 85.2%
Validation accuracy: 55.6%
Test accuracy: 61.1%
Epoch: 0007 cost= 418.473931885
Minibatch accuracy: 91.4%
Validation accuracy: 57.0%
Test accuracy: 63.0%
Epoch: 0009 cost= 412.245755005
Minibatch accuracy: 93.8%
Validation accuracy: 59.0%
Test accuracy: 64.7%
Optimization Finished!


**Answer: ** Extreme overfitting case is still overfitting.

---
Problem 4
---------

Try to get the best performance you can using a multi-layer model! The best reported test accuracy using a deep network is [97.1%](http://yaroslavvb.blogspot.com/2011/09/notmnist-dataset.html?showComment=1391023266211#c8758720086795711595).

One avenue you can explore is to add multiple layers.

Another one is to use learning rate decay:

    global_step = tf.Variable(0)  # count the number of steps taken.
    learning_rate = tf.train.exponential_decay(0.5, global_step, ...)
    optimizer = tf.train.GradientDescentOptimizer(learning_rate).minimize(loss, global_step=global_step)
 
 ---


In [12]:
dropout = 0.75
n_hidden_1 = 1024
n_hidden_2 = 1024
display_step = 2
training_epochs = 10
batch_size = 128
n_input = 784 


def multilayer_perceptron(x, weights, biases):
    # Hidden layer with RELU activation
    layer_1 = tf.add(tf.matmul(x, weights['h1']), biases['b1'])
    layer_1 = tf.nn.relu(layer_1)
    # Hidden layer with RELU activation
    layer_2 = tf.add(tf.matmul(layer_1, weights['h2']), biases['b2'])
    layer_2 = tf.nn.relu(layer_2)
    # Output layer with linear activation
    out_layer = tf.matmul(layer_2, weights['out']) + biases['out']
    return out_layer

def multilayer_perceptron_dropout(x, weights, biases):  
    layer_1 = tf.add(tf.matmul(x, weights['h1']), biases['b1'])
    layer_1 = tf.nn.relu(layer_1)
    layer_1 = tf.nn.dropout(layer_1, dropout)
    layer_2 = tf.add(tf.matmul(layer_1, weights['h2']), biases['b2'])
    layer_2 = tf.nn.relu(layer_2)
    layer_1 = tf.nn.dropout(layer_2, dropout)
    out_layer = tf.matmul(layer_2, weights['out']) + biases['out']
    return out_layer

graph = tf.Graph()
with graph.as_default():

  global_step = tf.Variable(0)  # count the number of steps taken.
  learning_rate = tf.train.exponential_decay(0.05, global_step, 1000, 0.7)

  # Input data. For the training data, we use a placeholder that will be fed
  # at run time with a training minibatch.
  tf_train_dataset = tf.placeholder(tf.float32,
                                    shape=(batch_size, image_size * image_size))
  tf_train_labels = tf.placeholder(tf.float32, shape=(batch_size, num_labels))
  tf_valid_dataset = tf.constant(valid_dataset)
  tf_test_dataset = tf.constant(test_dataset)
    

  weights = {
      'h1': tf.Variable(tf.random_normal([n_input, n_hidden_1])),
      'h2': tf.Variable(tf.random_normal([n_hidden_1, n_hidden_2])),
      'out': tf.Variable(tf.random_normal([n_hidden_2, num_labels]))
  }
  biases = {
      'b1': tf.Variable(tf.random_normal([n_hidden_1])),
      'b2': tf.Variable(tf.random_normal([n_hidden_2])),
      'out': tf.Variable(tf.random_normal([num_labels]))
  }
  pred = multilayer_perceptron_dropout(tf_train_dataset, weights, biases)
  # Define loss and optimizer
  cost = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(logits=pred, labels=tf_train_labels)+ 0.001*tf.nn.l2_loss(weights['out'])+0.001*tf.nn.l2_loss(weights['h1'])+ 0.001*tf.nn.l2_loss(weights['h2']))
  optimizer = tf.train.AdamOptimizer(learning_rate=learning_rate).minimize(cost, global_step = global_step)
    
  train_prediction = tf.nn.softmax(pred)
  valid_prediction = tf.nn.softmax(multilayer_perceptron(tf_valid_dataset, weights, biases))
  test_prediction = tf.nn.softmax(multilayer_perceptron(tf_test_dataset, weights, biases))


In [13]:
with tf.Session(graph=graph) as sess:
    tf.global_variables_initializer().run()
    # Training cycle
    for epoch in range(training_epochs):
        avg_cost = 0.
        num_steps = train_labels.shape[0]//batch_size
        # Loop over all batches
        for step in range(num_steps):
            offset = (step * batch_size) % (train_labels.shape[0] - batch_size)
            # Generate a minibatch.
            batch_data = train_dataset[offset:(offset + batch_size), :]
            batch_labels = train_labels[offset:(offset + batch_size), :]
    
            # Run optimization op (backprop) and cost op (to get loss value)
            _, c, prediction = sess.run([optimizer, cost, train_prediction], feed_dict={tf_train_dataset: batch_data,
                                                          tf_train_labels: batch_labels})
            # Compute average loss
            avg_cost += c / num_steps
        # Display logs per epoch step
        if epoch % display_step == 0:
            print("Epoch:", '%04d' % (epoch+1), "cost=", \
                "{:.9f}".format(avg_cost))
            print("Minibatch accuracy: %.1f%%" % accuracy(prediction, batch_labels))
            print("Validation accuracy: %.1f%%" % accuracy(valid_prediction.eval(), valid_labels))
            print("Test accuracy: %.1f%%" % accuracy(test_prediction.eval(), test_labels))
    print("Optimization Finished!")


Epoch: 0001 cost= 757.115956813
Minibatch accuracy: 60.2%
Validation accuracy: 52.6%
Test accuracy: 59.7%
Epoch: 0003 cost= 35.017873477
Minibatch accuracy: 85.9%
Validation accuracy: 83.6%
Test accuracy: 90.0%
Epoch: 0005 cost= 3.862782827
Minibatch accuracy: 88.3%
Validation accuracy: 85.5%
Test accuracy: 91.6%
Epoch: 0007 cost= 1.499445079
Minibatch accuracy: 87.5%
Validation accuracy: 87.0%
Test accuracy: 93.0%
Epoch: 0009 cost= 0.923606261
Minibatch accuracy: 89.1%
Validation accuracy: 88.4%
Test accuracy: 93.9%
Optimization Finished!
